# Training and Converting the Handwriting Recogn. Model

The model is first built and trained on the original dataset - MNIST dataset. The link is given below. <br>
The saved model is then converted to tflite with post-integer quanitzation. <br>
The EdgeTPU requires a fully qantized model with 8 bit integer data for input/output tensors, activations etc.<br> <br>
This file can be executed on a Windows/MacOS or Linux PC. The .tflite model file is then to be transferred to <br> 
EdgeTPU for inferencing

## Building and Training the Model


### Importing the neccessary libaries and the data set
[NIST, hand written digit data set](https://www.tensorflow.org/datasets/catalog/mnist)

In [ ]:
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import time, os
from pathlib import Path
from datetime import datetime

import numpy as np
from matplotlib.pyplot import imshow
from PIL import Image, ImageOps
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"]="4"
logdir=os.getcwd()+'/logs';
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1,write_graph=True, write_images=True,profile_batch = '500,520')

### Retrieving the training dataset
For information on the data set see here [http://yann.lecun.com/exdb/mnist/](http://yann.lecun.com/exdb/mnist/)

In [ ]:
(img_train, label_train), (img_test, label_test) = tfds.as_numpy(tfds.load('mnist',split=['train', 'test'],batch_size=-1,as_supervised=True,try_gcs=True))

### Processing the Data

In [ ]:
# Number of used data sets 60.000 = all
number_of_used_ds = 60000
# Number of classes; 0 to 9
num_classes = 10

scale.fit(img_train[0:number_of_used_ds].reshape(-1,784))

X_train_sc = scale.transform(img_train[0:number_of_used_ds,:,:,0].reshape(-1,28*28))
X_test_sc = scale.transform(img_test.reshape(-1,28*28))

y_train = tf.keras.utils.to_categorical(label_train[0:number_of_used_ds],10)
y_test = tf.keras.utils.to_categorical(label_test,10)

# Show an example picture
imshow(X_train_sc[1,].reshape(28,28),cmap='gray')
print(y_train[0:10,],X_train_sc.shape)

### Creating a neural network
Two experiments:

1. create a neuronal network only with fully connected layers, no convolutional layers, use 2 hidden layers with 50 neurons; **change the variable network to 'flatten'**
2. now create a convolutional neuronal network with the following layers (**change the variable network to 'conv2d'**):
|Layer (type)                 |  # filter / size  |
|-----------------------------|-------------------|
|conv2d (Conv2D)              |  20 (3,3)         |
|max_pooling2d (MaxPooling2D) |  - (2,2)          |
|conv2d_1 (Conv2D)            |  10 (3,3)         |
|max_pooling2d_1 (MaxPooling2 |  - (2,2)          |
|flatten (Flatten)            |  -                |
|dropout (Dropout)            |  -                |
|dense (Dense)                |  num_classes      |

Use the documentation available from here: [https://keras.io/api/layers/](https://keras.io/api/layers/)

In [ ]:
network = 'conv2d'
if network == 'conv2d':
    X_train=np.expand_dims((X_train_sc.reshape(-1,28,28)), -1)
    X_test=np.expand_dims((X_test_sc.reshape(-1,28,28)), -1)    
    input_shape = (28,28,1)
else: 
    X_train = X_train_sc
    X_test = X_test_sc
    input_shape = (28*28,)
#
model = keras.Sequential([
        keras.Input(shape=input_shape),
#-------------------------------------------------------------
        #layers.Dense(50, activation='relu',name='first_hidden'),
        #layers.Dense(50, activation='relu',name='second_hidden'),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(16, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
#-------------------------------------------------------------
        layers.Dense(num_classes, activation="softmax",name='output'),])
model.summary()

In [ ]:
start = time.time()
model.compile(loss="mean_squared_error", optimizer="sgd" ,metrics=["accuracy"])
model.fit(X_train, y_train, batch_size=200, epochs=5)
print("Duration %.2f " % (time.time() - start))

### Score on test data

In [ ]:
score = model.evaluate(X_test[0:200,], y_test[0:200,], verbose=0)
print("Test accuracy: %.2f %%" % (score[1]*100))

### Saving the Model

In [ ]:
tf.saved_model.save(model, 'saved_model')

# an alternative, usually for keras model:
#model.save('path')

### Loading the Model

In [ ]:
# run the below code if you want to load the model, not required here
model = load_model('saved_model')
# loading keras model:
#model = keras.models.load_model('path/to/location')

## Testing it with own hand written digit

In [ ]:
img = Image.open('../test.png').convert("L")
imshow(ImageOps.invert(img),cmap='gray')
test = scale.transform(np.array(ImageOps.invert(img).resize((28,28))).reshape(1,-1))
if network == 'conv2d':
    test = test.reshape(1,28,28,1)
else: 
    test = test.reshape(1,28*28)
#
# Predicting the Test set results
for i in range(0,num_classes):
   print("Detected a %.0f with %.2f %% " % (i,model.predict(test).reshape(-1,1)[i]*100))

## Converting the Model

### Creating Representative Dataset for Quantization

In [ ]:
# formatting and broadcasting the training dataset to match the model input size
img_train = img_train.reshape(img_train.shape[0], 28, 28, 1)
tf.cast(img_train, tf.float32)
print(img_train.shape)

# creating representative dataset generator function to quantize the model variables optimally
def representative_data_gen():
    # take 1 batch only of a portion of the dataset
    for input_value in tf.data.Dataset.from_tensor_slices(img_train).batch(1).take(100):
        yield [tf.cast(input_value, tf.float32)]

### Constructing the Converter and Converting the Model

In [ ]:
# building converter using the saved model
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model/')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.allow_custom_ops = True

# Ensuring that if any ops can't be quantized, the converter throws an error
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Setting the input and output tensors to uint8
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()
print('conversion successful')

The below code block contains some variations for converting the model (commented out). <br>
These variations are useful in cases of errors in conversion from the above code block. <br>
Typically the above code should work for most of the models.

In [ ]:
# some variations in convereter code for different models (commented out)
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
#converter.allow_custom_ops = True
#converter._experimental_new_quantizer = True

# freezing the input tensor for quantization
#imp_model = tf.saved_model.load('<path>')
#concrete_func = imp_model.signatures["serving_default"]
#concrete_func.inputs[0].set_shape(<shape>)
#converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func], imp_model)

### Saving the Converted Tflite Model

In [ ]:
# Save the model as .tflite file
with open('handwriting_model_quant.tflite', 'wb') as f:
  f.write(tflite_model_quant)
print('model saved successfully')

Netron <link> https://netron.app/ </link> can be used to visualize the quantized model: Load the model file into the app and it will show the entire model graph for easier analysis. <br> If the model is successfully quantized, you will see two 'quantize' blocks one after the input and the other before the output.

### Compiling the Model

The quantized tflite model has to be compiled by the edgetpu compiler to run on the Edge TPU. <br>
The compiler can be downloaded on a linux system and the below code line can be executed with this link <link>https://coral.ai/docs/edgetpu/compiler/#system-requirements</link> <br>
For non-linux based system, use the Colab here <link>https://colab.research.google.com/github/google-coral/tutorials/blob/master/compile_for_edgetpu.ipynb</link> <br>
A file with the same name but ending with 'edgetpu.tflite' will be created. <br> Transfer this file to Edge TPU for inferencing.

Alternatively, open the notebook on Edge TPU itself run the below line. Further explanation can be found in document 2.

In [ ]:
# compile the model
!edgetpu_compiler -s handwriting

Note: It is very important to check the result of compilation or the log to verify whether all the operations (or most) are mapped to Edge TPU or not. <br> Operations which are not mapped to Edge TPU will run on the CPU leading to a decline in performance. 